In [1]:
import pandas as pd
import numpy as np
import pickle
import spacy
import scispacy
from collections import Counter
import pathlib
from spacytextblob.spacytextblob import SpacyTextBlob
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from negspacy.negation import Negex
from negspacy.termsets import termset
ts = termset("en_clinical")

# Intantiate the spacy program specifying the model you want to use
nlp = spacy.load("en_core_sci_lg")

# Add the abbreviation pipe to the spacy pipeline.
nlp.add_pipe("abbreviation_detector")
# Add the hpo entity linker pipe to the spacy pipeline.
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True,
                                        "linker_name": "umls",
                                        'threshold':0.95,
                                        'max_entities_per_mention':1
                                       })
# add the sentiment analysis step
nlp.add_pipe('spacytextblob')
# add the negation detection
nlp.add_pipe("negex")
ts.add_patterns({
            "pseudo_negations": [],
            "termination": [],
            "preceding_negations": ["was no", "were no", "were never", "showed no", "a negative"],
            "following_negations": ["was negative", "remained negative", "was undetectable"],
        })

# nlp.remove_pipe("scispacy_linker")

/home/jcampbell/miniconda3/envs/gopher/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/jcampbell/miniconda3/envs/gopher/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [6]:
df = pickle.load(open('./ctrl_spacy_hpo_df.p', 'rb'))

In [7]:
df.head()

,pmid,pmcid,title,abstract,authors,journal,pub_type,pub_date,doi,issn,...,pmc_tgz,xml_parse_d,html_parse_d,pdf_parse_d,plain_parse_d,content_text,hpo_ents,hpo_triggers,neg_hpo_ents,neg_hpo_triggers
index,,,,,,,,,,,,,,,,,,,,,
bd105957999a4fb4a2357b1ed7649649,27734554,NaN,Revisiting acute normovolemic hemodilution and...,BACKGROUND: The majority of allogeneic transfu...,"[Sebastian R, Ratliff T, Winch PD, Tumin D, Go...",Paediatric anaesthesia,"[Journal Article, Observational Study]",2017-01-25,10.1111/pan.13014,1460-9592 (Electronic) 1155-5645 (Linking),...,0,{},{},{'file_path': './output/formats/pdfs/bd1059579...,{},Revisiting acute normovolemic hemodilution and...,"[Peripheral edema, Blue discoloration of the s...","[peripheral edema, Cyanosis]","[Increased mean corpuscular volume, Increased ...","[increased, increased, hemoglobin, hemoglobin,..."
62287ef8b7cb42d8a699c6b2fc6ee7a2,20664371,NaN,Psychological distress after severe trauma: a ...,BACKGROUND: The aim of the study was to invest...,"[Toien K, Myhren H, Bredal IS, Skogstad L, San...",The Journal of trauma,[Journal Article],2010-12-25,10.1097/TA.0b013e3181e125f3,1529-8809 (Electronic) 0022-5282 (Linking),...,0,{},{},{},{},ABS: BACKGROUND: The aim of the study was to i...,[],[],"[Underactive thyroid, Anxiousness, Depression,...","[low, Anxiety, Depression, anxiety, depression]"
ffe3f1b084464c279995b2b13039778e,30173563,NaN,Retrospective Review of Clinical and Chest X-R...,INTRODUCTION: We performed a retrospective stu...,"[Niles D, Larsen B, Balaji A, Delaney D, Campo...",Clinical pediatrics,[Journal Article],2018-12-25,10.1177/0009922818795902,1938-2707 (Electronic) 0009-9228 (Linking),...,0,{'file_path': './output/formats/xmls/ffe3f1b08...,{'file_path': './output/formats/htmls/ffe3f1b0...,{},{},INTRODUCTION: We performed a retrospective stu...,"[Collapsed lung, Pneumomediastinum, Abnormal e...","[pneumothorax, pneumomediastinum, abnormal]","[Clinical course, Intensity, Intensity, Fever,...","[clinical course, severity, Severity, fever, c..."
a6e476d89d9f47159af1e33eaf3e8766,30513050,NaN,Homeward Bound: A Case Series of Cross-Cultura...,"For most families, the preferred location of d...","[Thorvilson MJ, Manahan AJ, Schiltz BM, Collur...",Journal of palliative medicine,"[Case Reports, Journal Article]",2019-04-25,10.1089/jpm.2018.0301,1557-7740 (Electronic) 1557-7740 (Linking),...,0,{},{},{},{},"ABS: For most families, the preferred location...",[],[],[],[]
d0ec384909724d7d840f7968ddd6f959,29510663,PMC5838875,Burden of severe RSV disease among immunocompr...,BACKGROUND: Respiratory syncytial virus (RSV) ...,"[Chatzis O, Darbre S, Pasquier J, Meylan P, Ma...",BMC infectious diseases,"[Journal Article, Multicenter Study, Observati...",2018-03-06,10.1186/s12879-018-3002-3,1471-2334 (Electronic) 1471-2334 (Linking),...,1,{'file_path': './output/formats/xmls/d0ec38490...,{'file_path': './output/formats/htmls/d0ec3849...,{'file_path': './output/formats/pdfs/d0ec38490...,{},BACKGROUND: Respiratory syncytial virus (RSV) ...,[],[],"[Severe, Respiratory infections, Lower respira...","[Severe, respiratory tract infection, LRTI, pn..."


In [4]:
nlp.max_size = 2000000

In [5]:
paed_path = '/home/jcampbell/paed_covid_case_reports/case_report_retrieval/output/retrieved_df/retrieved_df2.p'
df = pickle.load(open(paed_path, 'rb'))   
condition = [type(text) == str for text in df['content_text']]
df=pd.DataFrame(df.loc[condition,:])

In [6]:
doc = nlp(df['content_text'][0])

/home/jcampbell/miniconda3/envs/gopher/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/jcampbell/miniconda3/envs/gopher/lib/python3.8/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [7]:
# # abbreviations
# abrvs = {'long_forms':[],'spans':[]}
# for abrv in doc._.abbreviations:
#     abrvs['long_forms'].append(abrv._.long_form)
#     abrvs['spans'].append((abrv.start,abrv.end))
    
# Counter(abrvs['long_forms'])

In [5]:
# Counter([str(ent) for ent in doc.ents])['']

In [7]:
# # abbreviations
# abrvs = {'long_forms':[],'spans':[]}
# for abrv in doc._.abbreviations:
#     abrvs['long_forms'].append(abrv._.long_form)
#     abrvs['spans'].append((abrv.start,abrv.end))
    
# Counter(abrvs['long_forms'])

In [6]:
# for abrv in doc._.abbreviations:
#     if str(abrv._.long_form) == 'chronic granulomatous disease':
#         print(abrv.sent)
#         print(len(str(abrv.sent)), '\n')


In [8]:


# # text = 'I had a really horrible day. It was the worst day ever! But every now and then I have a really good day that makes me happy.'
# doc = nlp(df['content_text'][1])
# for sent in doc.sents:
#     polarity = sent._.polarity
#     if polarity < -0.5:
#         print(sent)
#         print(sent._.assessments)
# # ent = doc.ents[10]
# # print(ent)
# # # doc._.polarity      # Polarity: -0.125
# # # doc._.subjectivity  # Sujectivity: 0.9
# # # doc._.assessments   # Assessments: [(['really', 'horrible'], -1.0, 1.0, None), (['worst', '!'], -1.0, 1.0, None), (['really', 'good'], 0.7, 0.6000000000000001, None), (['happy'], 0.8, 1.0, None)]
# # ent._.subjectivity

In [9]:
# from negspacy.termsets import termset
# ts = termset("en_clinical")
# print(ts.get_patterns())
# ts.add_patterns({
#             "pseudo_negations": [],
#             "termination": [],
#             "preceding_negations": ["was no", "were no", "were never", "showed no", "a negative"],
#             "following_negations": ["was negative", "remained negative", "was undetectable"],
#         })

In [10]:

# # from negspacy.negation import Negex

# doc = nlp(df['content_text'][2])
# # nlp = spacy.load("en_core_web_sm")
# # nlp.add_pipe("negex")

# # doc = nlp("She does not like Steve Jobs but likes Apple products.")
# for e in doc.ents:
#     if e._.negex == True:
#         print(doc[e.start:e.end])
#         print(e.sent)
#         print('\n')
# #     print(e.text, e._.negex)

In [8]:
# doc.ents[2]

In [9]:

# d
# if entity._.kb_ents:
#     # if there is a "match/mention" you get a cui
#     mention = entity._.kb_ents[0]
#     if mention:
#         # then we get the cui details (mainly the name)
#         print(linker.kb.cui_to_entity[mention[0]])

In [10]:
# doc

In [1]:
# this is a dictionary of CUIs with all the sentences attributed to that cui.
cui_d = {}

def entity_processing(index, doc, cui_d):
    hpos = []
    ents = [ent for ent in doc.ents]
    ents.extend([abrv for abrv in doc._.abbreviations])
    for ent in ents:
        # try mapping to HPO
        linker = nlp.get_pipe("scispacy_linker")
        if ent._.kb_ents:
            # if there is a "match/mention" you get a cui
            # we have set a high threshold and only keep one so there is only an index 0
            mention = ent._.kb_ents[0]
            if mention:
                # then we get the cui details (mainly the name)
                trigger = doc[ent.start:ent.end].text
                hpo = linker.kb.cui_to_entity[mention[0]]
                if hpo is not None:
                    # the result is a sring that needs a bit of parsing
                    hpo = str(hpo).split('\n')[0].split(', ')
                    cui = hpo[0].replace('CUI: ','')
                    hpo_str = hpo[1].replace('Name: ','')
                    # we then store attributes for each entity starting with HPO CUI
                    hpos.append(hpo_str)
                    # now we want to create a file for the cui or add to an existing one.
                    if cui_d.get(cui) == None:
                        cui_d.update({cui:{'indexes':[index],
                                           'hpo_str':hpo_str,
                                          'sents':[ent.sent.text],
                                          'negation':[ent._.negex],
                                          'polarity':[ent.sent._.polarity],
                                          'subj':[ent.sent._.subjectivity],
                                          'threshold':[mention[1]],
                                          'triggers':[trigger]}})
                    else:
                        cui_d[cui]['indexes'].append(index)
                        cui_d[cui]['sents'].append(ent.sent.text)
                        cui_d[cui]['negation'].append(ent._.negex)
                        cui_d[cui]['polarity'].append(ent.sent._.polarity)
                        cui_d[cui]['subj'].append(ent.sent._.subjectivity)
                        cui_d[cui]['threshold'].append(mention[1])
                        cui_d[cui]['triggers'].append(trigger)
    return hpos, cui_d


In [11]:
hpos, cui_d = entity_processing(df.index[0], doc, cui_d)

In [12]:
cui_df = pd.DataFrame.from_dict(cui_d, orient = 'index')
cui_df.head()

,indexes,hpo_str,sents,negation,polarity,subj,threshold,triggers
C1707883,[e1d20cf546fb43f9a82016dabdd95518],Editor,[To the Editor: Severe acute respiratory syndr...,[False],[0.265],[0.5016666666666667],[0.9999999403953552],[Editor]
C0205082,"[e1d20cf546fb43f9a82016dabdd95518, e1d20cf546f...",Severe (severity modifier),[To the Editor: Severe acute respiratory syndr...,"[False, False, False, False, False, False, False]","[0.265, 0.2375, 0.16666666666666666, 0.1666666...","[0.5016666666666667, 0.6375, 0.583333333333333...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[Severe, severe, severe, severe, severe, sever..."
C5203676,"[e1d20cf546fb43f9a82016dabdd95518, e1d20cf546f...",SARS-CoV-2,[To the Editor: Severe acute respiratory syndr...,"[False, False, False, True, False]","[0.265, 0.22727272727272727, 0.227272727272727...","[0.5016666666666667, 0.5454545454545454, 0.545...","[1.0, 1.0, 1.0, 1.0, 1.0]","[SARS-CoV-2, SARS-CoV-2, SARS-CoV-2, SARS-CoV-..."
C0549178,[e1d20cf546fb43f9a82016dabdd95518],Continuous,[To the Editor: Severe acute respiratory syndr...,[False],[0.265],[0.5016666666666667],[1.0],[ongoing]
C0205246,[e1d20cf546fb43f9a82016dabdd95518],Generalized,[To the Editor: Severe acute respiratory syndr...,[False],[0.265],[0.5016666666666667],[1.0],[global]


In [13]:
pickle.dump(cui_df, open('./cui_df.p', 'wb'))

In [18]:
cui_df = pickle.load(open('./cui_df.p', 'rb'))

In [17]:
# cui_df['sents'][9]

In [14]:
hpo_ents = {}

hpos = []
# iterate through each entity in the doc
for abrv in doc._.abbreviations:
    
    
    # try mapping to HPO
    linker = nlp.get_pipe("scispacy_linker")
    if abrv._.kb_ents:
        # if there is a "match/mention" you get a cui
        mention = abrv._.kb_ents[0]
        if mention:
            # then we get the cui details (mainly the name)
            print(doc[abrv.start:abrv.end])
            print(abrv._.long_form)
            hpo = linker.kb.cui_to_entity[mention[0]]
            if hpo is not None:
                print(hpo)
                # the result is a sring that needs a bit of parsing
                hpo = str(hpo).split('\n')[0].split(', ')
 
                # we then store attributes for each entity starting with HPO CUI
                hpos.append({'cui':hpo[0].replace('CUI: ',''),
                             # then we add the primary long form name
                             'name':hpo[1].replace('Name: ',''),
                             # then we add the start and end point of the entity (span)
                             'span':(entity.start, entity.end),
                             # then the score given for the confidence of the match
                             'hpo_thresh':mention[1],
                             # negation of the entity bool
                             'negation':entity._.negex,
                             # the sentiment for the sentence the entity is from
                             'polarity': entity.sent._.polarity,
                             # if the sentence is mostly opinion or fact
                             'subjectivity':entity.sent._.subjectivity
                            })
# now we add the ent_d list to our df_d               
hpo_ents.update({1:hpos})

CGD
chronic granulomatous disease
CUI: C0018203, Name: Chronic granulomatous disease
Definition: A defect of leukocyte function in which phagocytic cells ingest but fail to digest bacteria, resulting in recurring bacterial infections with granuloma formation. When chronic granulomatous disease is caused by mutations in the CYBB gene, the condition is inherited in an X-linked recessive pattern. When chronic granulomatous disease is caused by CYBA, NCF1, NCF2, or NCF4 gene mutations, the condition is inherited in an autosomal recessive pattern.
TUI(s): T047
Aliases (abbreviated, total: 17): 
	 Chronic granulomatous disease, NOS, CHRONIC GRANULOMATOUS DISEASE, granulomatous chronic disease, Granulomatous Diseases, Chronic, Chronic granulomatous disease, chronic granulomatous disease, CGD, Chronic Granulomatous Disease, chronic disease granulomatous, Granulomatous Disease, Chronic
CGD
chronic granulomatous disease
CUI: C0018203, Name: Chronic granulomatous disease
Definition: A defect of l

In [13]:
hpo_ents = {}

hpos = []
# iterate through each entity in the doc
for entity in doc.ents:
    # try mapping to HPO
    linker = nlp.get_pipe("scispacy_linker")
    if entity._.kb_ents:
        # if there is a "match/mention" you get a cui
        mention = entity._.kb_ents[0]
        if mention:
            # then we get the cui details (mainly the name)
            print(doc[entity.start:entity.end])
            hpo = linker.kb.cui_to_entity[mention[0]]
            if hpo is not None:
                print(hpo)
                # the result is a sring that needs a bit of parsing
                hpo = str(hpo).split('\n')[0].split(', ')
 
                # we then store attributes for each entity starting with HPO CUI
                hpos.append({'cui':hpo[0].replace('CUI: ',''),
                             # then we add the primary long form name
                             'name':hpo[1].replace('Name: ',''),
                             # then we add the start and end point of the entity (span)
                             'span':(entity.start, entity.end),
                             # then the score given for the confidence of the match
                             'hpo_thresh':mention[1],
                             # negation of the entity bool
                             'negation':entity._.negex,
                             # the sentiment for the sentence the entity is from
                             'polarity': entity.sent._.polarity,
                             # if the sentence is mostly opinion or fact
                             'subjectivity':entity.sent._.subjectivity
                            })
# now we add the ent_d list to our df_d               
hpo_ents.update({1:hpos})

Editor
CUI: C1707883, Name: Editor
Definition: A person who edits material for publication; a person having managerial and sometimes policy-making responsibility for the editorial part of a publishing firm or of a newspaper, magazine, or other publication; the supervisor or conductor of a department of a newspaper, magazine, etc.
TUI(s): T090
Aliases: (total: 0): 
	 
Severe
CUI: C0205082, Name: Severe (severity modifier)
Definition: Intensely bad or unpleasant in degree, quality or extent.
TUI(s): T033
Aliases (abbreviated, total: 16): 
	 gravy, Severe (severity modifier), grade high, High grade, gravies, high grade, Severely, Gravis, severe, severely
SARS-CoV-2
CUI: C5203676, Name: SARS-CoV-2
Definition: None
TUI(s): T005
Aliases: (total: 4): 
	 Wuhan coronavirus, 2019-nCoV, 2019 novel coronavirus, 2019 novel coronavirus (organism)
ongoing
CUI: C0549178, Name: Continuous
Definition: Remain in force or carry on without letup; keep or maintain in unaltered condition; exist in time or sp

In [3]:
hpo_ents = {}
for counter, doc in enumerate(nlp.pipe(list(df['content_text']), batch_size =100)):
    # every doc will get a list entities that map to HPO
    # each entity will have a dictionary with a series of values
    hpos = []
    # iterate through each entity in the doc
    for entity in doc.ents:
        # try mapping to HPO
        linker = nlp.get_pipe("scispacy_linker")
        if entity._.kb_ents:
            # if there is a "match/mention" you get a cui
            mention = entity._.kb_ents[0]
            if mention:
                # then we get the cui details (mainly the name)
                hpo = linker.kb.cui_to_entity[mention[0]]
                if hpo is not None:
                    # the result is a sring that needs a bit of parsing
                    hpo = str(hpo).split('\n')[0].split(', ')
                    # we then store attributes for each entity starting with HPO CUI
                    hpos.append({'cui':hpo[0].replace('CUI: ',''),
                                 # then we add the primary long form name
                                 'name':hpo[1].replace('Name: ',''),
                                 # then we add the start and end point of the entity (span)
                                 'span':(entity.start, entity.end),
                                 # then the score given for the confidence of the match
                                 'hpo_thresh':mention[1],
                                 # negation of the entity bool
                                 'negation':entity._.negex,
                                 # the sentiment for the sentence the entity is from
                                 'polarity': entity.sent._.polarity,
                                 # if the sentence is mostly opinion or fact
                                 'subjectivity':entity.sent._.subjectivity
                                })
    # now we add the ent_d list to our df_d               
    hpo_ents.update({df.index[counter]:hpos})
    if counter % 10 == 0:
        print(f'{counter} of {len(df)} complete')
# for each hpo we want the cui, name and threshold




/home/jcampbell/miniconda3/envs/gopher/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/jcampbell/miniconda3/envs/gopher/lib/python3.8/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


0 of 1672 complete
10 of 1672 complete
20 of 1672 complete
30 of 1672 complete
40 of 1672 complete
50 of 1672 complete
60 of 1672 complete
70 of 1672 complete
80 of 1672 complete
90 of 1672 complete
100 of 1672 complete
110 of 1672 complete
120 of 1672 complete
130 of 1672 complete
140 of 1672 complete
150 of 1672 complete
160 of 1672 complete
170 of 1672 complete
180 of 1672 complete
190 of 1672 complete
200 of 1672 complete
210 of 1672 complete
220 of 1672 complete
230 of 1672 complete
240 of 1672 complete
250 of 1672 complete
260 of 1672 complete
270 of 1672 complete
280 of 1672 complete
290 of 1672 complete
300 of 1672 complete
310 of 1672 complete
320 of 1672 complete
330 of 1672 complete
340 of 1672 complete
350 of 1672 complete
360 of 1672 complete
370 of 1672 complete
380 of 1672 complete
390 of 1672 complete
400 of 1672 complete
410 of 1672 complete
420 of 1672 complete
430 of 1672 complete
440 of 1672 complete
450 of 1672 complete
460 of 1672 complete
470 of 1672 complete
480

In [68]:
def val_to_df(ent_list):
    cols = list(ent_list[0].keys())
    # holding lists
    cuis = []
    names = []
    spans =[]
    hpo_thresh = []
    negation = []
    polarity = []
    subjectivity = []
    for val in ent_list:
        cuis.append(val['cui'])
        names.append(val['name'])
        spans.append(val['span'])
        hpo_thresh.append(val['hpo_thresh'])
        negation.append(val['negation'])
        polarity.append(val['polarity'])
        subjectivity.append(val['subjectivity'])
    df = pd.DataFrame(data = {'cui':cuis,
                              'name':names,
                              'span':spans,
                              'hpo_thresh':hpo_thresh,
                              'negation':negation,
                              'polarity':polarity,
                              'subjectivity':subjectivity})
    return df

In [74]:
ent_df = val_to_df(hpo_ents[df.index[1]])
len(ent_df)

101

In [75]:
Counter(ent_df['name']).most_common(10)

[('Increased number of platelets in blood', 10),
 ('Abnormality of the cerebrospinal fluid', 10),
 ('Lymphocytopenia', 8),
 ('Severe', 7),
 ('Fever', 7),
 ('Low platelet count', 6),
 ('Low neutrophil count', 5),
 ('Irritability', 4),
 ('Bone infection', 4),
 ('Thrombocytopenia', 4)]

In [90]:
for d in hpos:
    if d['name'] == 'Severe':
        print(doc[d['span'][0]].sent)
        print('\n')

Diabetic ketoacidosis at the onset of T1DM was more frequent, and a higher percentage of these patients presented with a severe form.


Several studies reported a significant increase in the frequency of DKA present at the diagnosis of T1DM, with a higher percentage of the severe form of this metabolic complication [,,,,,].


The degree of severity of DKA was classified according to 2018 International Society for Paediatric and Adolescent Diabetes (ISPAD) guidelines: mild DKA (pH < ), moderate DKA (pH < ), severe DKA (pH < ).


In the pandemic group, a higher percent of DKA cases developed the severe form compared to the pre-pandemic group ( vs. , OR = , CI 95% = –, p = ). (, ).


Among the eight patients, seven presented DKA which was mild in three cases, moderate in three, and severe in one patient.


Moreover, a higher percentage of children presented a severe form of this metabolic complication.




In [4]:
pickle.dump(hpo_ents, open('./umls_ents.p', 'wb'))

In [79]:
create ent_sents

SyntaxError: invalid syntax (<ipython-input-79-bf186ad93750>, line 1)